##Train

###Import dependencies

In [ ]:
!pip3 install tensorflow-gpu==1.13.1
!pip3 install opencv-python
!pip3 install imageai --upgrade
!pip3 install tensorflow==2.5.0rc0
!pip3 install keras==2.5.0rc0

###Connect to google drive to get the data

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/MyDrive/School/DeepLearning/FinalProj

In [ ]:
!unzip apple_detection_dataset.zip

###Train the Model

In [ ]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="apple_dataset")
trainer.setTrainConfig(object_names_array=["apple", "damaged_apple"], batch_size=8, num_experiments=50, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

##Test

### Run th model on a given image

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection
import os

execution_path = os.getcwd()

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(detection_model_path="detection_model-ex-028--loss-8.723.h5")
detector.setJsonPath(configuration_json="apple_dataset/json/detection_config.json")
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image="image2.jpg", minimum_percentage_probability=60, output_image_path="image-new2.jpg")

numOfApple = 0
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])
    numOfApple+=1

### Display the output

In [ ]:
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow
image = cv2.imread("image2-new.jpg")
cv2_imshow(image)
print("Total number of apples in the image:", numOfApple)

##Our addition

### Crop the apples and save as new images

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
imageName = "image2"
im = Image.open(imageName + ".jpg")
i=1
for detection in detections:
  left=detection["box_points"][0]
  top=detection["box_points"][1]
  right=detection["box_points"][2]
  bottom=detection["box_points"][3]
  im1 = im.crop((left, top, right, bottom))
  #%matplotlib inline
  im1.save("cropped/" + imageName + "-" + detection["name"] + str(i) + ".png")
  i+=1
  

### Read each cropped image, convert to black&white accordig to the ripe color, calculate the ripe percentage

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("cropped") if isfile(join("cropped", f))]


In [ ]:
for f in  onlyfiles: 
  image = cv2.imread("cropped/" + f)  
  hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  
  #define upper and lower color values in HSV.
  weaker = np.array([0,0,100])
  stronger = np.array([10,255,255])
  
  mask = cv2.inRange(hsv, weaker, stronger) 

  #Show original image and result
  cv2_imshow(image)
  cv2_imshow(mask)
  size = np.size(mask)
  print("Ripe percentage:", cv2.countNonZero(mask)/size*100*1.5,"\n")